# Next Token Prediction Model

```{note}
如何训练一个工业级的大语言模型？其中的关键挑战和必要的背景知识又是哪些？我们的方法是一步步来，先从简单的模型和训练方法开始，然后逐步增加复杂度。<BR/>
作为开头，本节我们定义一个简单的用于预测下一个token的模型，使用fake的数据，用Pytorch CPU跑通训练流程。
```

## 模型定义

In [8]:
import torch.nn as nn

class SimpleNextTokenModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        
        # 1. Embedding 层
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # 2. MLP 层
        # 简单结构: Linear -> Activation -> Linear
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, embed_dim)
        )
        
        # 3. 输出层 (Linear)
        # 将维度从 embed_dim 映射回 vocab_size
        self.output_head = nn.Linear(embed_dim, vocab_size, bias=False)

    def forward(self, input_ids):
        """
        参数:
            input_ids: (batch_size, sequence_length)
        返回:
            logits: (batch_size, sequence_length, vocab_size)
        """
        # x shape: (batch_size, seq_len, embed_dim)
        x = self.embedding(input_ids)
        
        # x shape: (batch_size, seq_len, embed_dim)
        x = self.mlp(x)
        
        # logits shape: (batch_size, seq_len, vocab_size)
        logits = self.output_head(x)
        
        return logits

In [2]:
import torch

torch.manual_seed(42)

# 超参数
vocab_size = 1000
embed_dim = 256
hidden_dim = 1024

# 初始化模型
model = SimpleNextTokenModel(vocab_size, embed_dim, hidden_dim)
model

SimpleNextTokenModel(
  (embedding): Embedding(1000, 256)
  (mlp): Sequential(
    (0): Linear(in_features=256, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=256, bias=True)
  )
  (output_head): Linear(in_features=256, out_features=1000, bias=False)
)

## 训练数据

In [3]:
def generate_dummy_data(num_batches, batch_size, seq_len, vocab_size):
    """生成简单的随机数据"""
    data = []
    for _ in range(num_batches):
        # 随机生成数据
        batch = torch.randint(0, vocab_size, (batch_size, seq_len))
        data.append(batch)
    return data

In [4]:
seq_len = 128   # 序列长度
batch_size = 32
num_batches = 100 # 训练步数

# 注意：这里的 data 包含输入和目标，所以 seq_len + 1
train_data = generate_dummy_data(num_batches, batch_size, seq_len + 1, vocab_size)
len(train_data), train_data[0].shape

(100, torch.Size([32, 129]))

## 训练

### CrossEntropyLoss 简介

交叉熵损失（Cross Entropy Loss）衡量的是两个概率分布 $P$（真实分布）和 $Q$（预测分布）之间的差异。其公式为：

$$
H(P, Q) = -\sum_{x} P(x) \log Q(x)
$$

对于 Next Token Prediction 任务，真实分布 $P$ 是一个 **One-hot 向量**（只有真实的下一个 token 位置为 1，其余为 0）。假设真实 token 的索引是 $i$，那么公式简化为：

$$
Loss = - \log(Q(x_i))
$$

其中 $Q(x_i)$ 是模型预测该位置为真实 token 的概率（经过 Softmax 归一化）。

### 为什么大语言模型使用它？

1. **最大似然估计（MLE）**：最小化交叉熵等价于最大化正确 token 的预测概率，这符合我们希望模型“猜对”下一个词的目标。
2. **处理多分类问题**：LLM 的词表（Vocab Size）通常很大（数万到数十万），CrossEntropyLoss 能自然地处理这种大规模多分类问题。
3. **梯度特性好**：当模型预测错误（概率低）时，$-\log(p)$ 的梯度很大，能让模型迅速修正；当预测正确（概率接近 1）时，梯度趋近于 0，训练稳定。

In [5]:
import torch.optim as optim

# 定义 Loss 和 Optimizer
criterion = nn.CrossEntropyLoss()
learning_rate = 1e-3
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

In [6]:
import time

# 记录开始时间
start_time = time.time()
print(f"开始训练，共 {num_batches} 个 batch...")
model.train()

for step, batch in enumerate(train_data):
    # 构造输入和目标 (Next Token Prediction)
    # 输入: 序列的前 N-1 个 token
    # 目标: 序列的后 N-1 个 token (即每个位置的下一个 token)
    input_ids = batch[:, :-1]  # (B, T)
    targets = batch[:, 1:]     # (B, T)
    
    # 清零梯度
    optimizer.zero_grad()
    
    # 前向传播
    logits = model(input_ids) # (B, T, V)
    
    # 计算 Loss
    # CrossEntropyLoss 需要 (N, C) 和 (N) 的输入
    # logits.view(-1, vocab_size) -> (B*T, V)
    # targets.view(-1) -> (B*T)
    loss = criterion(logits.view(-1, vocab_size), targets.reshape(-1))
    
    # 反向传播
    loss.backward()
    
    # 更新参数
    optimizer.step()
    
    if (step + 1) % 10 == 0:
        print(f"Step [{step+1}/{num_batches}], Loss: {loss.item():.4f}")

end_time = time.time()
print(f"训练结束！耗时: {end_time - start_time:.2f} 秒")

开始训练，共 100 个 batch...
Step [10/100], Loss: 6.9229
Step [20/100], Loss: 6.9222
Step [30/100], Loss: 6.9219
Step [40/100], Loss: 6.9241
Step [50/100], Loss: 6.9204
Step [60/100], Loss: 6.9230
Step [70/100], Loss: 6.9176
Step [80/100], Loss: 6.9225
Step [90/100], Loss: 6.9193
Step [100/100], Loss: 6.9180
训练结束！耗时: 7.11 秒


## 推理

In [7]:
# 简单验证一下
test_input = train_data[0][:, :-1]
with torch.no_grad():
    logits = model(test_input)
    preds = torch.argmax(logits, dim=-1)
    print("\n验证第一个 batch 的预测:")
    print(f"输入 shape: {test_input.shape}")
    print(f"预测 shape: {preds.shape}")
    # 这里只是随机数据，预测准确率不会高，主要是验证流程跑通


验证第一个 batch 的预测:
输入 shape: torch.Size([32, 128])
预测 shape: torch.Size([32, 128])


## 

## 